In [1]:
import datetime
import pandas as pd
import requests
import json
import csv
import os

In [2]:
crypto_list = [
    "bitcoin",
    "ethereum",
    "binancecoin",
    "cardano",
    "solana",
    "polkadot",
    "avalanche-2",
    "chainlink",
    "matic-network",
    "vechain"
]

base_url = "https://api.coingecko.com/api/v3/coins"
crypto_data = []

for crypto in crypto_list:
    url = f"{base_url}/{crypto}/market_chart?vs_currency=usd&days=max"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        if data and 'prices' in data:
            first_price_data = data['prices'][0]
            timestamp = first_price_data[0] / 1000
            date = datetime.datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d')
            crypto_data.append({"crypto": crypto, "date": date})
        else:
            crypto_data.append({"crypto": crypto, "date": "No data available"})
    else:
        crypto_data.append({"crypto": crypto, "date": "Error fetching data"})

# Crear df fdesde crypto_data
df = pd.DataFrame(crypto_data)

print(df)

# Convertir la columna 'date' al formato de marca de tiempo UNIX sin hora, minuto y segundo
df['from_timestamp'] = pd.to_datetime(df['date']).dt.floor('D').astype('int64') // 10 ** 9

print(df)

# Agregar la columna 'to_timestamp' con la marca de tiempo UNIX del día de hoy
today_timestamp = int(datetime.datetime.today().replace(hour=0, minute=0, second=0, microsecond=0).timestamp())
df['to_timestamp'] = today_timestamp

# Eliminar la columna 'date'
df.drop(columns=['date'], inplace=True)

          crypto        date
0        bitcoin  2013-04-28
1       ethereum  2015-08-07
2    binancecoin  2017-09-16
3        cardano  2017-10-18
4         solana  2020-04-11
5       polkadot  2020-08-19
6    avalanche-2  2020-09-22
7      chainlink  2017-11-09
8  matic-network  2019-04-27
9        vechain  2018-07-27
          crypto        date  from_timestamp
0        bitcoin  2013-04-28      1367107200
1       ethereum  2015-08-07      1438905600
2    binancecoin  2017-09-16      1505520000
3        cardano  2017-10-18      1508284800
4         solana  2020-04-11      1586563200
5       polkadot  2020-08-19      1597795200
6    avalanche-2  2020-09-22      1600732800
7      chainlink  2017-11-09      1510185600
8  matic-network  2019-04-27      1556323200
9        vechain  2018-07-27      1532649600


In [3]:
df

,crypto,from_timestamp,to_timestamp
0,bitcoin,1367107200,1692759600
1,ethereum,1438905600,1692759600
2,binancecoin,1505520000,1692759600
3,cardano,1508284800,1692759600
4,solana,1586563200,1692759600
5,polkadot,1597795200,1692759600
6,avalanche-2,1600732800,1692759600
7,chainlink,1510185600,1692759600
8,matic-network,1556323200,1692759600
9,vechain,1532649600,1692759600


In [4]:
# Ruta al directorio donde se guardarán los archivos CSV
output_directory = "csv_loop"

# Crear el directorio si no existe
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

#Funcion para extraer los datos de la Api
def extract_Crypto(crypto, from_timestamp, to_timestamp):
    # Construir la URL para la criptomoneda específica
    url = f"https://api.coingecko.com/api/v3/coins/{crypto}/market_chart/range?vs_currency=usd&from={from_timestamp}&to={to_timestamp}&price=price&market_cap=market_cap&volume=volume"

    response = requests.get(url)
    data = response.json()

    # Obtener los valores relevantes del JSON
    timestamps = [entry[0] for entry in data["prices"]]
    market_caps = [entry[1] for entry in data["market_caps"]]
    prices = [entry[1] for entry in data["prices"]]
    volumes = [entry[1] for entry in data["total_volumes"]]

    # Verificar que todas las listas tengan la misma longitud
    min_length = min(len(timestamps), len(market_caps), len(prices), len(volumes))
    timestamps = timestamps[:min_length]
    market_caps = market_caps[:min_length]
    prices = prices[:min_length]
    volumes = volumes[:min_length]

    data_dict = {
        "Timestamp": timestamps,
        "Market Cap": market_caps,
        "Price": prices,
        "Volume": volumes
    }
    df = pd.DataFrame(data_dict)

    print(f"df_{crypto} creado")
    return df


# df_coin = extract_Crypto(crypto, from_timestamp, to_timestamp)

def to_CSV(output_directory, df):
    # Ruta completa al archivo CSV en el directorio de salida
    csv_file_path = os.path.join(output_directory, f"{crypto}.csv")

    # Guardar el DataFrame en un archivo CSV
    df.to_csv(csv_file_path, index=False)

    print(f"Archivo {crypto}.csv creado en el directorio {output_directory}.")
    return


# Aplicar extract_Crypto a cada fila y guardar en archivos CSV
for index, row in df.iterrows():
    crypto = row["crypto"]
    from_timestamp = row["from_timestamp"]
    to_timestamp = row["to_timestamp"]

    # Aplicar extract_Crypto a la fila actual y obtener el DataFrame resultante
    extracted_df = extract_Crypto(crypto, from_timestamp, to_timestamp)

    # Guardar el DataFrame en un archivo CSV
    to_CSV(output_directory, extracted_df)

df_bitcoin creado
Archivo bitcoin.csv creado en el directorio csv_loop.
df_ethereum creado
Archivo ethereum.csv creado en el directorio csv_loop.
df_binancecoin creado
Archivo binancecoin.csv creado en el directorio csv_loop.
df_cardano creado
Archivo cardano.csv creado en el directorio csv_loop.
df_solana creado
Archivo solana.csv creado en el directorio csv_loop.
df_polkadot creado
Archivo polkadot.csv creado en el directorio csv_loop.
df_avalanche-2 creado
Archivo avalanche-2.csv creado en el directorio csv_loop.
df_chainlink creado
Archivo chainlink.csv creado en el directorio csv_loop.
df_matic-network creado
Archivo matic-network.csv creado en el directorio csv_loop.
df_vechain creado
Archivo vechain.csv creado en el directorio csv_loop.


In [ ]:
# Directorio que contiene los archivos CSV
directory = 'csv_loop/'

# Obtén una lista de todos los archivos CSV en el directorio
csv_files = [file for file in os.listdir(directory) if file.endswith('.csv')]

# Itera a través de cada archivo CSV
for file in csv_files:
    # Carga el archivo CSV en un DataFrame
    file_path = os.path.join(directory, file)
    df = pd.read_csv(file_path)
    
    # Convierte la columna 'Timestamp' y crea la columna 'Date'
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], unit='ms')
    df['Date'] = df['Timestamp'].dt.strftime('%Y-%m-%d')
    
    # Elimina la columna original 'Timestamp'
    df.drop('Timestamp', axis=1, inplace=True)
    
    # Guarda el DataFrame modificado de nuevo en el archivo CSV
    df.to_csv(file_path, index=False)